In [2]:
import requests
from bs4 import BeautifulSoup
from datetime import date, timedelta
import time
import pandas as pd
import re

In [3]:
# Cria dataframe para receber o conteúdo
Campos = ['Data',
           'Alagamento',
           'Bairro',
           'Tipo',
           'Horário Inicial',
           'Horário Final',
           'Endereço',
           'Sentido',
           'Referência'
           ]
df = pd.DataFrame(columns=Campos)

In [4]:
# Função para determinar se houve alagamento ou não em um determinado dia
def houve_alag(soup):
    # Texto padrão quando não houve alagamentos
    texto = 'Não há registros de alagamentos para essa data.'

    # Variável para alagamento. 0, sem alagamento. 1, com alagamento
    flag_alag = 0

    alag = soup.find_all(string = re.compile('registros'))
    
    if (alag == []):
        flag_alag = 1
        return flag_alag
    else:
        flag_alag = 0
        return flag_alag

In [6]:
# Montagem dinâmica das datas a serem varridas:
start = date(2023, 1, 1)
end = date(2023, 1, 5)
# Quando a função estiver totalmente funcional
# end = date.today()
data_pesquisada = start
dia = timedelta(days=1)

index= 0

# Inicia o preenchimento do dataframe
while data_pesquisada < end:
    data_formatada = data_pesquisada.strftime('%m')+'%2F'+data_pesquisada.strftime('%d')+'%2F'+data_pesquisada.strftime('%Y')
    URL_pesquisada = 'https://www.cgesp.org/v3/alagamentos.jsp?dataBusca='+data_formatada+'&enviaBusca=Buscar'
    print(URL_pesquisada)

    # Faz a requisição para a página
    try:
        response = requests.get(URL_pesquisada)
    except requests.exceptions.Timeout:
        print('Timeout informado pelo sistema')
    except requests.exceptions.TooManyRedirects:
        print('Problema na formação da URL. Favor tentar com outra')
    except requests.exceptions.RequestException as e:
        print('Erro catastrófico')
        raise SystemExit(e)

    # Analisa o HTML da página com o BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
        
    if (houve_alag(soup)==0):
        print(f"Em {data_pesquisada} não houve registros de alagamentos.")
        df.at[index,'Data'] =  data_pesquisada
        df.at[index,'Alagamento'] = 0
        df.at[index,'Bairro'] = 0
        df.at[index,'Tipo'] = 0
        df.at[index,'Horário Inicial'] = 0
        df.at[index,'Horário Final'] = 0
        df.at[index,'Endereço'] = 0
        df.at[index,'Sentido'] = 0
        df.at[index,'Referência'] = 0
        index = index +1 
    else:
        for pontos in soup.find_all('td', {'class':'total-pts arial-bairros-alag'}):
            texto = pontos.string
            no_alag = int(texto[0:texto.find('pt')-1])
            zona = soup.find_all('h1',{'class':'tit-bairros'})

            # Vamos iterar sobre a quantidade de pontos de alagamento
            for pt_alag in range(no_alag):
            
                df.at[index,'Data'] =  data_pesquisada
                df.at[index,'Alagamento'] = 1

                pai = pontos.parent
            
                linhas = soup.find_all('div',{'class':'ponto-de-alagamento'})
            
                df.at[index,'Bairro'] = pai.find('td',{'class':'bairro arial-bairros-alag linha-pontilhada'}).text.strip()
                df.at[index,'Tipo'] = linhas[pt_alag].find('li')['title']
            
                #'Horário Inicial',
                texto = linhas[pt_alag].find('li', {'class':'arial-descr-alag col-local'}).text
                horas =re.findall('(?:(\d\d?):)?([0-5][0-9])',texto)
                df.at[index,'Horário Inicial'] = str(horas[0][0])+':'+str(horas[0][1])

                #'Horário Final'
                df.at[index,'Horário Final'] = str(horas[1][0])+':'+str(horas[1][1])

                # Endereço
                df.at[index,'Endereço'] = texto[texto.find(horas[1][1])+2:len(texto)]

                #'Sentido',
                texto = linhas[pt_alag].find_all('li', {'class':'arial-descr-alag'})
                info = texto[1].get_text()
                df.at[index,'Sentido'] = info[8:info.find('Ref')]

                #'Referência'
                df.at[index,'Referência'] = info[info.find('Ref')+11:len(info)]
            
                index = index +1 
    data_pesquisada = data_pesquisada + dia
    time.sleep(1)
df

https://www.cgesp.org/v3/alagamentos.jsp?dataBusca=01%2F01%2F2023&enviaBusca=Buscar
https://www.cgesp.org/v3/alagamentos.jsp?dataBusca=01%2F02%2F2023&enviaBusca=Buscar
https://www.cgesp.org/v3/alagamentos.jsp?dataBusca=01%2F03%2F2023&enviaBusca=Buscar
Em 2023-01-03 não houve registros de alagamentos.
https://www.cgesp.org/v3/alagamentos.jsp?dataBusca=01%2F04%2F2023&enviaBusca=Buscar
Em 2023-01-04 não houve registros de alagamentos.


,Data,Alagamento,Bairro,Tipo,Horário Inicial,Horário Final,Endereço,Sentido,Referência
0,2023-01-01,1,Itaquera,Inativo Intransitável,14:16,14:48,AVENIDA ARICANDUVA,ITAQUERA/MARGINAL,AVENIDA AFONSO DE SAMPAIO E SOUZA
1,2023-01-01,1,Itaquera,Inativo Transitável,14:35,17:55,RUA PADRE VIEGAS DE MENEZES,UNICO,RUA ITAGIMIRIM
2,2023-01-02,1,Vila Maria/ Vila Guilherme,Inativo Intransitável,21:19,22:31,R SEM DENOMINACAO,BAIRRO/CENTRO,R DA BARACELA
3,2023-01-02,1,Sé,Inativo Intransitável,21:19,22:31,R SEM DENOMINACAO,BAIRRO/CENTRO,R DA BARACELA
4,2023-01-02,1,Lapa,Inativo Intransitável,21:19,22:31,R SEM DENOMINACAO,BAIRRO/CENTRO,R DA BARACELA
5,2023-01-02,1,Aricanduva/ Vila Formosa,Inativo Intransitável,21:19,22:31,R SEM DENOMINACAO,BAIRRO/CENTRO,R DA BARACELA
6,2023-01-02,1,Itaquera,Inativo Intransitável,21:19,22:31,R SEM DENOMINACAO,BAIRRO/CENTRO,R DA BARACELA
7,2023-01-02,1,Itaquera,Inativo Intransitável,06:25,08:46,R DOM RODO,AMBOS,R GUAPORE
8,2023-01-02,1,São Miguel Paulista,Inativo Intransitável,21:19,22:31,R SEM DENOMINACAO,BAIRRO/CENTRO,R DA BARACELA
9,2023-01-02,1,São Mateus,Inativo Intransitável,21:19,22:31,R SEM DENOMINACAO,BAIRRO/CENTRO,R DA BARACELA


In [395]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13 entries, 0 to 12
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Data             13 non-null     object
 1   Alagamento       13 non-null     object
 2   Bairro           13 non-null     object
 3   Tipo             13 non-null     object
 4   Horário Inicial  13 non-null     object
 5   Horário Final    13 non-null     object
 6   Endereço         13 non-null     object
 7   Sentido          13 non-null     object
 8   Referência       13 non-null     object
dtypes: object(9)
memory usage: 1.6+ KB


In [1]:
df.to_excel('Alagamentos_SP.xls',sheet_name='Alagamentos')

NameError: name 'df' is not defined

In [ ]:
#URL = 'https://www.cgesp.org/v3/alagamentos.jsp?dataBusca=08%2F03%2F2023&enviaBusca=Buscar'
#URL = 'https://www.cgesp.org/v3/alagamentos.jsp?dataBusca=07%2F03%2F2023&enviaBusca=Buscar'

# Faz a requisição para a página
#response = requests.get(URL)
    
# Analisa o HTML da página com o BeautifulSoup
#soup = BeautifulSoup(response.content, 'html.parser')